In [1]:
import pandas as pd
import numpy as np
pd.options.mode.chained_assignment = None

In [2]:
df = pd.read_table("all_table_definitions.txt", delimiter="|", encoding= 'unicode_escape')

In [3]:
df.rename(columns=lambda x: x.strip(), inplace=True)

In [4]:
new = df[df['Field Name'].str.contains('Table     :')]['Field Name'].str.split(' ', expand=True)[6]
index= new.index.values
df1= df.copy()
df1['Table Name'] = np.NaN
df1.loc[index, ['Table Name']] = new.values
df1['Table Name'].fillna(method='ffill', inplace=True)
df1.head()

,Field Name,Description,Domain,Datatype Physic.,Mand.,Acti,Refer. Reference,Table Name
0,,...,,length,,,Table Mode,NaN
1,---------------+------------------------------...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1 indt,...,tcdate,UTC Date/T 4,No,Yes,Not applicab,NaN
3,2 dsca,...,tcmcs.str50m,Multibyte 200,No,Yes,Not applicab,NaN
4,3 wfrc,...,asint.wfld,0,No,Yes,Not applicab,NaN


In [5]:
new1 = df1[df1['Field Name'].str.contains('Delete Reference Message')]['Field Name'].str.split(' ', expand=True)[8]
index1= new.index.values
df2= df1.copy()
df2['VCR'] = np.NaN
df2.loc[index1, ['VCR']] = new1.values
df2['VCR'].fillna(method='ffill', inplace=True)
df3 = df2[df2['VCR'] == '107U0']
df3.head()

,Field Name,Description,Domain,Datatype Physic.,Mand.,Acti,Refer. Reference,Table Name,VCR
22,Table : asint000 ...,NaN,NaN,NaN,NaN,NaN,NaN,asint000,107U0
23,VRC : 107U0 a000 (Update) ...,NaN,NaN,NaN,NaN,NaN,NaN,asint000,107U0
24,...,NaN,NaN,NaN,NaN,NaN,NaN,asint000,107U0
25,----------------------------------------------...,NaN,NaN,NaN,NaN,NaN,NaN,asint000,107U0
26,Field Name,Description ...,Domain,Datatype Physic.,Mand.,Acti,Refer. Reference,asint000,107U0


In [6]:
table_name = ['tccom001', 'tcmcs002', 'tcmcs010', 'tcmcs015', 'tcmcs023', 'tiedm010', 'tiedm100', 'tiedm110']

In [7]:
appended_data = []
table_data = []
for name in table_name:
    try:
        newdf = df3[df3['Table Name'].str.contains(name)].dropna(subset=['Field Name','Description'])
        list_numbers = str([1, 2, 3, 4, 5, 6, 7, 8, 9])
        newdf1 = newdf[newdf['Field Name'].str.strip().astype(str).str.startswith(tuple(list_numbers))]
        newdf1['Field Name'] = newdf1['Field Name'].str.split(' ', expand=True, n=3)[3].str.strip()
        nan_value = float("NaN")
        newdf1.replace("", nan_value, inplace=True)
        newdf1.dropna(subset = ["Field Name"], inplace=True)
        newdf1['Description'] = newdf1['Description'].str.split('(', expand=True, n=1)[0].str.strip()
        newdf1['Datatype'] = newdf1['Datatype     Physic.'].str.split(' ',  expand=True, n=1)[1].str.split('  ',  expand=True, n=1)[0]
        newdf1['Physic.'] = newdf1['Datatype     Physic.'].str.split(' ',  expand=True, n=1)[1].str.split('  ',  expand=True, n=1)[1]
        appended_data.append(newdf1)
        table_data.append(name)
    except ValueError: 
        print("A ValueError occurred")
    except:
        print(name, "does not have any information" )
appended_data = pd.concat(appended_data)
appended_data.drop('Datatype     Physic.', axis=1, inplace=True)
print(table_data, "does have information")
appended_data

tcmcs015 does not have any information
['tccom001', 'tcmcs002', 'tcmcs010', 'tcmcs023', 'tiedm010', 'tiedm100', 'tiedm110'] does have information


,Field Name,Description,Domain,Mand.,Acti,Refer. Reference,Table Name,VCR,Datatype,Physic.
111763,emno,Employee,tcemno,Yes,Yes,Not applicab,tccom001,107U0,String,9
111764,nama,Name,tcnama,No,Yes,Not applicab,tccom001,107U0,Multibyte,140
111765,namb,Given Name,tccom.name,No,Yes,Not applicab,tccom001,107U0,Multibyte,200
111766,namc,Middle Name,tccom.name,No,Yes,Not applicab,tccom001,107U0,Multibyte,200
111767,namd,Last Name,tccom.name,No,Yes,Not applicab,tccom001,107U0,Multibyte,200
...,...,...,...,...,...,...,...,...,...,...
293570,usup,Use Up Materials Present,tcyesno,Yes,Yes,Not applicab,tiedm110,107U0,Enumerated,1
293571,rdsp,Reference Designators Present,tcyesno,Yes,Yes,Not applicab,tiedm110,107U0,Enumerated,1
293572,cmba,Combined E-Item & Revision & Positions,,,,Not applicab,tiedm110,107U0,,
293576,cmbb,Combined of E-Item & Revision,,,,tiedm100 Mandatory,tiedm110,107U0,,


In [8]:
appended_data.to_excel('test.xlsx')

In [9]:
databs = appended_data[['Field Name', 'Description','Table Name']]
databs['Field Name'] = 'T_' + databs['Field Name'].astype(str).str.upper()
newdata = databs[databs['Table Name'] == 'tcmcs002']
newdata


,Field Name,Description,Table Name
133973,T_CCUR,Currency,tcmcs002
133974,T_DSCA,Description,tcmcs002
133975,T_DSCB,Short Description,tcmcs002
133976,T_CRND,Rounding Factor,tcmcs002
133977,T_EMUC,EMU Currency,tcmcs002
133978,T_BREP,BLWI Reporting Currency,tcmcs002
133979,T_GTRF,Grand Total Rounding Factor,tcmcs002
133980,T_ICCC,ISO Currency Code,tcmcs002


In [10]:
newdata['Description'] = newdata['Description'].str.replace(" ", "_")
newdata['Name'] =  newdata['Field Name'] + ' AS '   + newdata['Description'].str.lower() + '_' +newdata['Field Name'].str.split('T_', expand=True)[1]
names = list(newdata['Name'])
new_name =   ',\n'.join(names) 
print(new_name)

T_CCUR AS currency_CCUR,
T_DSCA AS description_DSCA,
T_DSCB AS short_description_DSCB,
T_CRND AS rounding_factor_CRND,
T_EMUC AS emu_currency_EMUC,
T_BREP AS blwi_reporting_currency_BREP,
T_GTRF AS grand_total_rounding_factor_GTRF,
T_ICCC AS iso_currency_code_ICCC


In [13]:
ln_site = '100'
table_name = 'ttcmcs002'

source_dataset = 's-bicloud-curated-0001-p.ln_100'
destination_dataset = 's-finance-dev-0001-s.cons_LN'
  
query_text_start = f"CREATE OR REPLACE VIEW '{destination_dataset}.{table_name}{ln_site}' AS \n"
# We could append the human-readable site name to the table name but I couldn't find that list

final_query = query_text_start + f"SELECT\n{new_name}\nFROM '{source_dataset}.{table_name}{ln_site}'"
print(final_query)

CREATE OR REPLACE VIEW 's-finance-dev-0001-s.cons_LN.ttcmcs002100' AS 
SELECT
T_CCUR AS currency_CCUR,
T_DSCA AS description_DSCA,
T_DSCB AS short_description_DSCB,
T_CRND AS rounding_factor_CRND,
T_EMUC AS emu_currency_EMUC,
T_BREP AS blwi_reporting_currency_BREP,
T_GTRF AS grand_total_rounding_factor_GTRF,
T_ICCC AS iso_currency_code_ICCC
FROM 's-bicloud-curated-0001-p.ln_100.ttcmcs002100'
